### Implementing Data Validation using Protobuf in a Distributed System
**Description**: Use gRPC to implement a distributed system that validates messages using
Protobuf.

**Steps**:
1. Create a .proto file for gRPC service.
2. Implement server-side validation
    - Create a gRPC server
    - Bind the server to an address
    - Start server

In [3]:
from concurrent import futures
import grpc
import re
import time

import validation_pb2
import validation_pb2_grpc


def is_valid_email(email):
    pattern = r"^[\w\.-]+@[\w\.-]+\.\w+$"
    return re.match(pattern, email) is not None


class ValidatorService(validation_pb2_grpc.ValidatorServicer):
    def ValidatePerson(self, request, context):
        try:
            if not request.name:
                return validation_pb2.ValidationResult(is_valid=False, message="Name cannot be empty")
            if request.age < 0:
                return validation_pb2.ValidationResult(is_valid=False, message="Age must be non-negative")
            if not is_valid_email(request.email):
                return validation_pb2.ValidationResult(is_valid=False, message="Invalid email format")

            return validation_pb2.ValidationResult(is_valid=True, message="Validation successful")
        except Exception as e:
            context.set_code(grpc.StatusCode.INTERNAL)
            context.set_details(f"Server error: {str(e)}")
            return validation_pb2.ValidationResult(is_valid=False, message="Server encountered an error")


def serve():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    validation_pb2_grpc.add_ValidatorServicer_to_server(ValidatorService(), server)
    server.add_insecure_port('[::]:50051')
    print("Server started on port 50051.")
    server.start()
    try:
        while True:
            time.sleep(86400)
    except KeyboardInterrupt:
        print("Shutting down server...")
        server.stop(0)


if __name__ == '__main__':
    serve()


ModuleNotFoundError: No module named 'validation_pb2'